代码连接：https://github.com/Arifkhan21/Human-Pose-Estimation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
df=pd.read_csv("../input/mpii-dataset-for-python/dataset/mpii_dataset.csv")

In [ ]:
def imag(ix):
    img = df['NAME'][ix]
    print(img)
    im = Image.open('../input/mpii-dataset-for-python/dataset/images/'+img )
    print(im.size)
    implot = plt.imshow(im)
    px=2
    while px <34:
        col = df.columns[px]
        x = df[col][ix]
        col = df.columns[px+1]
        y = df[col][ix]
        print(col,x,y)
        plt.scatter([[x]],[[y]])
        px=px+2

In [ ]:
imag(1783)

In [ ]:
def pix1(ix):
    img = df['NAME'][ix]
    print(img)
    im = Image.open('../input/mpii-dataset-for-python/dataset/images/'+img )
    #implot=plt.imshow(im)
    #plt.show()
    im2=im.resize((224,224))
    implot = plt.imshow(im2)
    imsize=np.array(im.size)
    fact=imsize/224.0
    print(fact)
    px=2
    kx=ix
    while px <34:
        col = df.columns[px]
        x = df[col][ix]
        col = df.columns[px+1]
        y = df[col][ix]
        print(col,x,y)
        #plt.scatter([[x]],[[y]])
        px=px+2
    cx=2
    while cx<34:
        col=df.columns[cx]
        x=df[col][kx]/fact[0]
        col=df.columns[cx+1]
        y=df[col][kx]/fact[1]
        print(col,x,y)
        plt.scatter([[x]],[[y]])
        cx+=2
    plt.show()

In [ ]:
pix1(18)

In [ ]:

import os
import cv2
import h5py
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from PIL import Image
from pathlib import Path
from keras.models import Sequential,Model
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16,preprocess_input

from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Input,Flatten,SeparableConv2D,GlobalAveragePooling2D
from keras.layers import GlobalMaxPool2D
from keras import backend as K
from keras.optimizers import Adam,SGD,RMSprop
from keras.callbacks import ModelCheckpoint,Callback,EarlyStopping,TensorBoard,ReduceLROnPlateau,CSVLogger,LearningRateScheduler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
color=sns.color_palette()
%matplotlib inline

In [ ]:
def step_decay(epoch):
    initial_rate=0.1
    drop=0.5
    epochs_drop=5.0
    lrate=intia_rate*math.pow(drop,math.floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:

train_data='../input/mpii-dataset-for-python/dataset/images/'

In [ ]:
data=pd.read_csv('../input/mpii-dataset-for-python/dataset/mpii_dataset.csv')
data=data[:4000]
print(type(data['NAME'].values))
image_data=[]
labels=[]
data
print(data.shape[0])
print(data['NAME'].shape)

In [ ]:
label_dict=dict((name,i) for i,name in enumerate(data['Category'].unique()))
name_dict=dict((i,name) for i,name in enumerate(data['Category'].unique()))
print(label_dict)
print(name_dict)

In [ ]:
import tqdm
for img_path in data['NAME'].values:
    img=load_img('../input/mpii-dataset-for-python/dataset/images/'+img_path,target_size=(224,224))
    img_array=img_to_array(img)
    image_data.append(img_array)

In [ ]:
image_data2=np.asarray(image_data)

In [ ]:
y_train=data[data.columns[2:-3]]
print(y_train.columns)
print(y_train.shape)
y_train2=np.ones((y_train.shape))
print(y_train.shape)
for ix in range(data.shape[0]):
    img=df['NAME'][ix]
    im = Image.open('../input/mpii-dataset-for-python/dataset/images/'+img )
    #implot=plt.imshow(im)
    #plt.show()
    im2=im.resize((224,224))
    #implot = plt.imshow(im2)
    imsize=np.array(im.size)
    fact=imsize/224.0
    cx=2
    while cx<34:
        col=df.columns[cx]
        x=df[col][ix]/fact[0]
        col=df.columns[cx+1]
        y=df[col][ix]/fact[1]
        #print(col,x,y)
        y_train2[ix][cx-2]=x
        y_train2[ix][cx-1]=y
        cx+=2

In [ ]:
from keras.applications.resnet50 import ResNet50
X=image_data2    #include_top=False,
base_model=ResNet50(include_top=False,input_shape=(224,224,3),weights='imagenet')
for layer in base_model.layers[:-7]:
    layer.trainable=False
# for layer in base_model.layers:
#     print(layer,layer.trainable)
print(base_model.summary())

In [ ]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
# model.add(Dense(1024,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation='linear'))
model.summary()

In [ ]:
checkpoint=ModelCheckpoint('./base.model',
                          monitor='val_loss',
                          verbose=1,
                          save_best_only=True,
                          mode='max',
                          save_weights_only=False,
                          period=1)
earlystop=EarlyStopping(monitor='val_loss',
                       min_delta=0.001,
                       patience=30,
                       verbose=1,
                       mode='auto')
tensorboard = TensorBoard(
    log_dir = './logs',
    histogram_freq=0,
    batch_size=16,
    write_graph=True,
    write_grads=True,
    write_images=False,
)

csvlogger = CSVLogger(
    filename= "training_csv.log",
    separator = ",",
    append = False
)

lrsched = LearningRateScheduler(step_decay,verbose=1)

reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=5,
    verbose=1, 
    mode='auto',
    min_delta=0.0001, 
    cooldown=1, 
    min_lr=0.0001
)
callbacks = [checkpoint,tensorboard,earlystop,csvlogger,reduce]

In [ ]:
X.shape,y_train2.shape

In [ ]:
from keras.optimizers import Adam

In [ ]:
# opt=Adam(lr=1e-2)
opt=SGD(lr=1e-5,momentum=0.95)
# model = EfficientNetB0.build(width=224, height=224, channel=3, classes=32)
# model = MobileNetV2_TL(input_shape, 32)
model.compile(loss='mean_absolute_error',
             optimizer=opt,metrics=['mean_absolute_error'])
hist=model.fit(X,y_train2,epochs=20,shuffle=True,batch_size=2,validation_split=0.1,callbacks=callbacks)

In [ ]:
def show_final_history(history):
    fig,ax=plt.subplots(1,2,figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch,history.history['loss'],label="Train Losss")
    ax[0].plot(history.epoch,history.history["val_loss"],label="Validation_loss")
    ax[1].set_title('mse')
    ax[1].plot(history.epoch,history.history["mean_absolute_error"],label="Train_acc")
    ax[1].plot(history.epoch,history.history["val_mean_absolute_error"],label="validaton_loss")
    ax[0].legend()
    ax[1].legend()
show_final_history(hist)

In [ ]:
y_pred=model.predict(X[190:191])

In [ ]:
n=191
img = df['NAME'][n]
print(img)
im = load_img('../input/mpii-dataset-for-python/dataset/images/'+img,target_size=(224,224))
img = img_to_array(im)
img_ = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
print(img.shape,img_.shape)
points = y_pred[0]
print(len(points))
i=0
plt.imshow(im)
while i < 16:
    x = points[2*i]
    y = points[(2*i)+1]
    plt.scatter([[x]],[[y]])
    i=i+1

In [ ]:
# base_model=VGG16(include_top=False,
#                 input_shape=(224,224,3),
#                 weights='imagenet')
# for layer in base_model.layers[:-8]:
#     layer.trainable=False
# for layer in base_model.layers:
#     print(layer,layer.trainable)
# print(base_model.summary())

In [ ]:
# model=Sequential()
# model.add(base_model)
# model.add(Flatten())
# model.add(Dense(1024,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(128,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(32,activation='linear'))
# model.summary()

In [ ]:
# checkpoint=ModelCheckpoint('./base.model',
#                           monitor='val_loss',
#                           verbose=1,
#                           save_best_only=True,
#                           mode='max',
#                           save_weights_only=False,
#                           period=1)
# earlystop=EarlyStopping(monitor='val_loss',
#                        min_delta=0.001,
#                        patience=30,
#                        verbose=1,
#                        mode='auto')
# tensorboard = TensorBoard(
#     log_dir = './logs',
#     histogram_freq=0,
#     batch_size=16,
#     write_graph=True,
#     write_grads=True,
#     write_images=False,
# )

# csvlogger = CSVLogger(
#     filename= "training_csv.log",
#     separator = ",",
#     append = False
# )

# lrsched = LearningRateScheduler(step_decay,verbose=1)

# reduce = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.8,
#     patience=5,
#     verbose=1, 
#     mode='auto',
#     min_delta=0.0001, 
#     cooldown=1, 
#     min_lr=0.0001
# )

# callbacks = [checkpoint,tensorboard,earlystop,csvlogger,reduce]

In [ ]:
# opt=SGD(lr=1e-5,momentum=0.95)
# model.compile(loss='mean_squared_error',
#              optimizer=opt,metrics=['mean_squared_error'])

In [ ]:
# history=model.fit(image_data2,y_train2,steps_per_epoch=10,
#                            validation_split=0.2,
#                            validation_steps=10,
#                            epochs=100,verbose=1,
#                            callbacks=callbacks)

In [ ]:
# show_final_history(history)
# model_json = model.to_json()
# with open("tmodel.json","w") as json_file:
#     json_file.write(model_json)
# model.save("tmodel.h5")

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image
# import cv2

In [ ]:
# df=pd.read_csv("../input/mpii-dataset-for-python/dataset/mpii_dataset.csv")

In [ ]:
# img = cv2.imread('../input/mpii-dataset-for-python/dataset/images/000025245.jpg')
# gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
# print(img.shape)
# print(gray.shape)
# plt.imshow(gray)
# imsize=np.array(img.size)
# fact=imsize/224.0
# print(fact)

In [ ]:
# def pix1(ix):
#     img = df['NAME'][ix]
#     print(img)
#     im = Image.open('../input/mpii-dataset-for-python/dataset/images/'+img)
#     img =cv2.imread('../input/mpii-dataset-for-python/dataset/images/'+img)
#     img_c = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     res = cv2.resize(img_c,(224,224))
#     #implot=plt.imshow(im)
#     #plt.show()
#     im2=im.resize((224,224))
# #     implot = plt.imshow(im2)
#     implot2 = plt.imshow(res)
#     imsize=np.array(im.size)
#     fact=imsize/224.0
#     print(fact)
    
#     px=2
#     kx=ix
#     while px <34:
#         col = df.columns[px]
#         x = df[col][ix]
#         col = df.columns[px+1]
#         y = df[col][ix]
#         print(col,x,y)
#         #plt.scatter([[x]],[[y]])
#         px=px+2
#     cx=2
#     while cx<34:
#         col=df.columns[cx]
#         x=df[col][kx]/fact[0]
#         col=df.columns[cx+1]
#         y=df[col][kx]/fact[1]
#         print(col,x,y)
#         plt.scatter([[x]],[[y]])
#         cx+=2
#     plt.show()

In [ ]:
# pix1(178)

In [ ]:
# import os
# import cv2
# import h5py
# import keras
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from PIL import Image
# from pathlib import Path
# from keras.models import Sequential,Model
# from keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16,preprocess_input
# from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
# from keras.layers import Convolution2D,MaxPooling2D,Dense,Dropout,Input,Flatten,SeparableConv2D,GlobalAveragePooling2D
# from keras.layers import GlobalMaxPool2D
# from keras import backend as K
# from keras.optimizers import Adam,SGD,RMSprop
# from keras.callbacks import ModelCheckpoint,Callback,EarlyStopping,TensorBoard,ReduceLROnPlateau,CSVLogger,LearningRateScheduler
# from keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import confusion_matrix
# color=sns.color_palette()
# %matplotlib inline

In [ ]:
# def step_decay(epoch):
#     initial_rate=0.1
#     drop=0.5
#     epochs_drop=5.0
#     lrate=intia_rate*math.pow(drop,math.floor((1+epoch)/epochs_drop))
#     return lrate

In [ ]:
# def show_final_history(history):
#     fig,ax=plt.subplots(1,2,figsize=(15,5))
#     ax[0].set_title('loss')
#     ax[0].plot(history.epoch,history.history['loss'],label="Train Losss")
#     ax[0].plot(history.epoch,history.history["val_loss"],label="Validation_loss")
#     ax[1].set_title('mse')
#     ax[1].plot(history.epoch,history.history["mean_absolute_error"],label="Train_acc")
#     ax[1].plot(history.epoch,history.history["val_mean_absolute_error"],label="validaton_loss")
#     ax[0].legend()
#     ax[1].legend()

In [ ]:
# train_data='../input/mpii-dataset-for-python/dataset/images/'

In [ ]:

# # data=pd.read_csv('mpii_dataset.csv')
# data=df[:7000]
# print(type(data['NAME'].values))
# print(data.shape[0])
# print(data['NAME'].shape)
# data.head()

In [ ]:
# cnt=0
# image_data=[]
# labels=[]
# for img_path in data['NAME'].values:
#     cnt=cnt+1
#     img=cv2.imread('../input/mpii-dataset-for-python/dataset/images/'+img_path)
#     img_c = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     gray = cv2.cvtColor(img_c,cv2.COLOR_RGB2GRAY)
#     res = cv2.resize(gray,(224,224))
#     img_array=img_to_array(img)
#     image_data.append(res)


In [ ]:
# print(len(image_data))
# image_data2=np.asarray(image_data)
# print(image_data2.shape)
# arr = np.array(gray.shape)
# print(arr)

zhuy8i

In [ ]:
# y_train=data[data.columns[2:-3]]
# print(y_train.columns)
# print(y_train.shape)
# y_train2=np.ones((y_train.shape))
# print(y_train.shape)
# for ix in range(data.shape[0]):
#     img=df['NAME'][ix]
#     im = Image.open('../input/mpii-dataset-for-python/dataset/images/'+img )
#     #implot=plt.imshow(im)
#     #plt.show()
#     im2=im.resize((224,224))
#     #implot = plt.imshow(im2)
#     imsize=np.array(im.size)
#     fact=imsize/224.0
#     cx=2
#     while cx<34:
#         col=df.columns[cx]
#         x=df[col][ix]/fact[0]
#         col=df.columns[cx+1]
#         y=df[col][ix]/fact[1]
#         #print(col,x,y)
#         y_train2[ix][cx-2]=x
#         y_train2[ix][cx-1]=y
#         cx+=2

In [ ]:
# y_train=data[data.columns[2:-3]]
# cnt=0
# y_train2=np.ones((y_train.shape))
# for ix in range(data.shape[0]):
#     img=df['NAME'][ix]
#     im = load_img('../input/mpii-dataset-for-python/dataset/images/'+img)
#     img = img_to_array(im)    
# #     img_c = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
# #     res = cv2.resize(gray,(224,224))
#     #implot=plt.imshow(im)
#     #plt.show()
#     im2=im.resize((224,224))
#     #implot = plt.imshow(im2)
#     imsize=np.array(im.size)
#     fact=imsize/224.0
# #     print(fact)
#     cx=2
#     while cx<34:
#         col=df.columns[cx]
#         x=df[col][ix]/fact[0]
#         col=df.columns[cx+1]
#         y=df[col][ix]/fact[1]
#         #print(col,x,y)
#         y_train2[ix][cx-2]=x
#         y_train2[ix][cx-1]=y
#         cx+=2
#     cnt = cnt+1

In [ ]:
# y_train2=np.asarray(y_train2)
# y_train2 = y_train2.astype(np.float32)

In [ ]:
# n=190
# img = df['NAME'][n]
# im = load_img('../input/mpii-dataset-for-python/dataset/images/'+img,target_size=(224,224))
# img = img_to_array(im)
# img_ = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
# points = y_train2[n]
# print(points)
# i=0
# plt.imshow(img_)
# while i < 16:
#     x = points[2*i]
#     y = points[(2*i)+1]
#     plt.scatter([[x]],[[y]])
#     i=i+1

111

In [ ]:
from keras.applications.resnet50 import ResNet50
X=image_data2    #include_top=False,
base_model=ResNet50(include_top=False,input_shape=(224,224,3),weights='imagenet')
for layer in base_model.layers[:-7]:
    layer.trainable=False
# for layer in base_model.layers:
#     print(layer,layer.trainable)
print(base_model.summary())

111

In [ ]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
# model.add(Dense(1024,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation='linear'))
model.summary()

In [ ]:
# from keras import backend
# from keras.layers import Conv2D
# from keras.layers import BatchNormalization
# from keras.layers import Reshape
# from keras.layers import Dropout
# from keras.layers import multiply
# from keras.layers import Activation
# from keras.layers import DepthwiseConv2D
# from keras.layers import Add
# from keras.layers import Input
# from keras.layers import GlobalAveragePooling2D
# from keras.models import Model
# from keras.layers import Dense
# from keras.utils import get_custom_objects


# class EfficientNetB0():
#     def swish(x):
#         return backend.sigmoid(x) * x

#     get_custom_objects().update({'swish': Activation(swish)})

#     @staticmethod
#     def build(width, height, channel, classes):

#         input_layer = Input(shape=(width, height, channel),
#                             name="input")
#         # Stage1
#         # 卷积
#         x = Conv2D(filters=32,
#                    kernel_size=(3, 3),
#                    strides=(2, 2),
#                    padding='same',
#                    activation='swish',
#                    name='s1_conv_1_7x7/2')(input_layer)

#         x = BatchNormalization(name="s1_bn",
#                                momentum=0.99,
#                                epsilon=1e-5,
#                                )(x)

#         x = Activation(activation='swish', name='s1_swish')(x)

#         # Stage2
#         x = i_se_res(x, 16, stride=1, expansion=1, ksize=3, block_id=0, stage='s2')

#         # Stage3
#         x = i_se_res(x, 24, stride=2, expansion=6, ksize=3, block_id=1, stage='s3')
#         x = i_se_res(x, 24, stride=1, expansion=6, ksize=3, block_id=2, stage='s3')

#         # Stage4
#         x = i_se_res(x, 40, stride=2, expansion=6, ksize=5, block_id=3, stage='s4')
#         x = i_se_res(x, 40, stride=1, expansion=6, ksize=5, block_id=4, stage='s4')

#         # Stage5
#         x = i_se_res(x, 80, stride=2, expansion=6, ksize=3, block_id=5, stage='s5')
#         x = i_se_res(x, 80, stride=1, expansion=6, ksize=3, block_id=6, stage='s5')
#         x = i_se_res(x, 80, stride=1, expansion=6, ksize=3, block_id=7, stage='s5')

#         # Stage6
#         x = i_se_res(x, 112, stride=1, expansion=6, ksize=5, block_id=8, stage='s6')
#         x = i_se_res(x, 112, stride=1, expansion=6, ksize=5, block_id=9, stage='s6')
#         x = i_se_res(x, 112, stride=1, expansion=6, ksize=5, block_id=10, stage='s6')

#         # Stage7
#         x = i_se_res(x, 192, stride=2, expansion=6, ksize=5, block_id=11, stage='s7')
#         x = i_se_res(x, 192, stride=1, expansion=6, ksize=5, block_id=12, stage='s7')
#         x = i_se_res(x, 192, stride=1, expansion=6, ksize=5, block_id=13, stage='s7')
#         x = i_se_res(x, 192, stride=1, expansion=6, ksize=5, block_id=14, stage='s7')

#         # Stage8
#         x = i_se_res(x, 320, stride=1, expansion=6, ksize=3, block_id=15, stage='s8')

#         # Stage9
#         # 卷积
#         x = Conv2D(filters=1280,
#                    kernel_size=(1, 1),
#                    strides=(1, 1),
#                    padding='same',
#                    use_bias=False,
#                    name='s9_conv_3_1x1/1')(x)

#         x = BatchNormalization(name="s9_bn",
#                                momentum=0.99,
#                                epsilon=1e-5,
#                                )(x)

#         x = Activation(activation='swish', name='s9_swish')(x)

#         x = GlobalAveragePooling2D(name='global_avg_pool')(x)

#         x = Dropout(0.2)(x)

#         x = Dense(classes, activation='softmax', name='output')(x)
#         return Model(input_layer, x, name='EfficientNetB0')


# def i_se_res(inputs, filters, stride=1, expansion=6, ksize=3, block_id=0, stage='s1'):

#     channel_axis = 1 if backend.image_data_format() == 'channel_first' else -1

#     in_channels = backend.int_shape(inputs)[channel_axis]

#     expansion_filters = expansion * in_channels

#     prefix = stage + '_' + 'b{}_'.format(block_id)

#     x = Conv2D(expansion_filters,
#                kernel_size=1,
#                padding='same',
#                use_bias=False,
#                activation=None,
#                name=prefix + 'expansion')(inputs)
#     x = BatchNormalization(axis=channel_axis,
#                            momentum=0.99,
#                            epsilon=1e-5,
#                            name=prefix + 'bn')(x)

#     x = Activation(activation='swish', name=prefix + 'swish')(x)

#     # 逐深卷积
#     x = DepthwiseConv2D(kernel_size=ksize,
#                         strides=stride,
#                         activation=None,
#                         use_bias=False,
#                         padding='same',
#                         name=prefix + 'conv_dw')(x)

#     x = BatchNormalization(axis=channel_axis,
#                            momentum=0.99,
#                            epsilon=1e-5,
#                            name=prefix + 'bn_dw')(x)

#     x = Activation(activation='swish', name=prefix + 'swish_dw')(x)

#     # target_shape = (1, 1, expansion_filters) if backend.image_data_format() == 'channel_list' else (
#     #     expansion_filters, 1, 1)
#     target_shape = (1,1,expansion_filters)
#     se = GlobalAveragePooling2D(name=prefix + "se_global_avg_pool")(x)

#     se = Reshape(target_shape, name=prefix + 'se_reshape')(se)

#     se = Conv2D(expansion_filters // 4,
#                 kernel_size=1,
#                 strides=1,
#                 activation='swish',
#                 padding='same',
#                 use_bias=True,
#                 name=prefix + "se_conv_1")(se)

#     se = Conv2D(expansion_filters,
#                 kernel_size=1,
#                 strides=1,
#                 activation='sigmoid',
#                 padding='same',
#                 use_bias=True,
#                 name=prefix + "se_conv_2")(se)
#     x = multiply([x, se], name=prefix + "se_multiply")

#     # 逐点卷积，降低维度
#     x = Conv2D(filters,
#                 kernel_size=1,
#                 padding='same',
#                 use_bias=False,
#                 activation=None,
#                 name=prefix + "conv_pw")(x)

#     x = BatchNormalization(axis=channel_axis,
#                            momentum=0.99,
#                            epsilon=1e-5,
#                            name=prefix + 'bn_pw')(x)

#     if inputs.shape[3] != filters:
#         inputs = Conv2D(filters,
#                         strides=stride,
#                         kernel_size=1,
#                         padding='same',
#                         use_bias=False,
#                         activation=None,
#                         name=prefix + 'residual_conv')(inputs)

#         inputs = BatchNormalization(axis=channel_axis,
#                                     momentum=0.99,
#                                     epsilon=1e-5,
#                                     name=prefix + 'residual_bn')(inputs)
#         inputs = Activation(activation='swish', name=prefix + 'residual_swish')(inputs)

#         x = Dropout(0.2)(x)

#     x = Add(name=prefix + 'add')([inputs, x])
#     return x
# # 测试
# if __name__ == "__main__":
#     model = EfficientNetB0.build(width=224, height=224, channel=3, classes=1000)
#     print(model.summary())

In [ ]:
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.layers import GlobalAveragePooling2D
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# def MobileNetV2_TL(input_shape=(224, 224, 3), classes=32):
#     model_base = MobileNetV2(include_top=False,
#                              weights='imagenet',
#                              input_shape=input_shape)
#     for layer in model_base.layers:
#         layer.trainable = True
#     model = Sequential()
#     model.add(model_base)
#     model.add(GlobalAveragePooling2D())
#     model.add(Dropout(0.5))
#     model.add(Dense(units=classes, activation='softmax'))
#     return model
# if __name__ == "__main__":
#     input_shape = (224, 224, 3)
#     model = MobileNetV2_TL(input_shape, 3)

In [ ]:
checkpoint=ModelCheckpoint('./base.model',
                          monitor='val_loss',
                          verbose=1,
                          save_best_only=True,
                          mode='max',
                          save_weights_only=False,
                          period=1)
earlystop=EarlyStopping(monitor='val_loss',
                       min_delta=0.001,
                       patience=30,
                       verbose=1,
                       mode='auto')
tensorboard = TensorBoard(
    log_dir = './logs',
    histogram_freq=0,
    batch_size=16,
    write_graph=True,
    write_grads=True,
    write_images=False,
)

csvlogger = CSVLogger(
    filename= "training_csv.log",
    separator = ",",
    append = False
)

lrsched = LearningRateScheduler(step_decay,verbose=1)

reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=5,
    verbose=1, 
    mode='auto',
    min_delta=0.0001, 
    cooldown=1, 
    min_lr=0.0001
)
callbacks = [checkpoint,tensorboard,earlystop,csvlogger,reduce]

In [ ]:
X.shape,y_train2.shape

In [ ]:
from keras.optimizers import Adam

In [ ]:
# opt=Adam(lr=1e-2)
opt=SGD(lr=1e-5,momentum=0.95)
# model = EfficientNetB0.build(width=224, height=224, channel=3, classes=32)
# model = MobileNetV2_TL(input_shape, 32)
model.compile(loss='mean_absolute_error',
             optimizer=opt,metrics=['mean_absolute_error'])
hist=model.fit(X,y_train2,epochs=100,shuffle=True,batch_size=2,validation_split=0.1,callbacks=callbacks)

In [ ]:
# from keras.layers import Convolution2D,MaxPooling2D,Dense,Dropout,Input,Flatten,SeparableConv2D,GlobalAveragePooling2D
# model1 = Sequential()
# model1.add(Convolution2D(32,(3,3),activation='relu', input_shape=(224,224,1)))
# model1.add(MaxPooling2D(2,2))
# model1.add(Dropout(0.1))
# model1.add(Convolution2D(64, (2,2), activation='relu'))
# model1.add(MaxPooling2D(2,2))
# model1.add(Dropout(0.25))
# model1.add(Convolution2D(128,(2,2),activation='relu'))
# model1.add(MaxPooling2D(2,2))
# model1.add(Dropout(0.5))
# model1.add(Flatten())
# model1.add(Dense(500,activation='relu'))
# model1.add(Dense(500,activation='relu'))
# model1.add(Dense(32,activation='linear'))

# model1.summary()

In [ ]:
# history=model.fit(X,y_train2,steps_per_epoch=50,
#                            validation_split=0.2,
#                            validation_steps=50,
#                            epochs=7,verbose=1,
#                            callbacks=callbacks)

In [ ]:
# from keras.layers import Convolution2D,MaxPooling2D,Dense,Dropout,Input,Flatten,SeparableConv2D,GlobalAveragePooling2D
# model1 = Sequential()
# model1.add(Convolution2D(32,(3,3),activation='relu', input_shape=(224,224,1)))
# model1.add(MaxPooling2D(2,2))
# model1.add(Dropout(0.1))
# model1.add(Convolution2D(64, (2,2), activation='relu'))
# model1.add(MaxPooling2D(2,2))
# model1.add(Dropout(0.25))
# model1.add(Convolution2D(128,(2,2),activation='relu'))
# model1.add(MaxPooling2D(2,2))
# model1.add(Dropout(0.5))
# model1.add(Flatten())
# model1.add(Dense(500,activation='relu'))
# model1.add(Dense(500,activation='relu'))
# model1.add(Dense(32,activation='linear'))

# model1.summary()
# model.compile(loss='mean_absolute_error',optimizer='adam',metrics=['mean_absolute_error'])

# hist1=model.fit(X,y_train2,epochs=20,shuffle=True,batch_size=2,validation_split=0.2,callbacks=callbacks)

In [ ]:
X.shape

In [ ]:
def show_final_history(history):
    fig,ax=plt.subplots(1,2,figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch,history.history['loss'],label="Train Losss")
    ax[0].plot(history.epoch,history.history["val_loss"],label="Validation_loss")
    ax[1].set_title('mse')
    ax[1].plot(history.epoch,history.history["mean_absolute_error"],label="Train_acc")
    ax[1].plot(history.epoch,history.history["val_mean_absolute_error"],label="validaton_loss")
    ax[0].legend()
    ax[1].legend()
show_final_history(hist)

In [ ]:
y_pred=model.predict(X[190:191])

In [ ]:
y_pred.shape

In [ ]:
y_pred[0],y_train2[191]

In [ ]:
n=191
img = df['NAME'][n]
print(img)
im = load_img('../input/mpii-dataset-for-python/dataset/images/'+img,target_size=(224,224))
img = img_to_array(im)
img_ = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
print(img.shape,img_.shape)
points = y_pred[0]
print(len(points))
i=0
plt.imshow(im)
while i < 16:
    x = points[2*i]
    y = points[(2*i)+1]
    plt.scatter([[x]],[[y]])
    i=i+1

In [ ]:
def pix1(ix):
    img = df['NAME'][ix]
    print(img)
    im = Image.open('../input/mpii-dataset-for-python/dataset/images/'+img )
    #implot=plt.imshow(im)
    #plt.show()
    im2=im.resize((224,224))
    implot = plt.imshow(im2)
    imsize=np.array(im.size)
    fact=imsize/224.0
    print(fact)
    px=2
    kx=ix
    while px <34:
        col = df.columns[px]
        x = df[col][ix]
        col = df.columns[px+1]
        y = df[col][ix]
        print(col,x,y)
        #plt.scatter([[x]],[[y]])
        px=px+2
    cx=2
    while cx<34:
        col=df.columns[cx]
        x=df[col][kx]/fact[0]
        col=df.columns[cx+1]
        y=df[col][kx]/fact[1]
        print(col,x,y)
        plt.scatter([[x]],[[y]])
        cx+=2
    plt.show()
pix1(3)